# 합성곱 신경망 데모

In this notebook, we use the batch, multi-channel convolution operation implemented in Numpy (that you can find [here](../lincoln/lincoln/conv.py)) to train a small convolutional neural network to more than 90% accuracy on MNIST.

이 노트북은 넘파이에 구현된 배치 학습 및 다채널 합성곱 연산을 

In [1]:
import sys
# 예제 파일 경로로 수정한 다음 주석 해제
# sys.path.append(r'/to/your/example_code/path/lincoln')
sys.path.append(r'/home/flourscent/DLFS_code/lincoln')

In [5]:
import numpy as np

import lincoln
from lincoln.layers import Dense
from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
from lincoln.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer
from lincoln.utils import mnist
from lincoln.layers import Conv2D

#mnist.init() # 최초 실행시 주석 해제, 이후 다시 주석 처리할 것

X_train, y_train, X_test, y_test = mnist.load()

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [8]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [9]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [14]:
def calc_accuracy_model(model, test_set):
    return print(f'''검증 데이터에 대한 모델의 정확도 : 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

# 밑바닥부터 구현한 CNN

In [12]:
model = NeuralNetwork(
    layers=[Conv2D(out_channels=16,
                   param_size=5,
                   dropout=0.8,
                   weight_init="glorot",
                   flatten=True,
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190402)

trainer = Trainer(model, SGDMomentum(lr = 0.1, momentum=0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs = 1,
            eval_every = 1,
            seed=20190402,
            batch_size=60,
            conv_testing=True);

0 배치 학습 후 손실값은  31.191501893742316
10 배치 학습 후 손실값은  14.15039049044619
20 배치 학습 후 손실값은  8.507022939163363
30 배치 학습 후 손실값은  9.816083122397819
40 배치 학습 후 손실값은  2.707136219766947
50 배치 학습 후 손실값은  5.039261518632477
60 배치 학습 후 손실값은  3.8410006567939794
70 배치 학습 후 손실값은  8.50500471291163
80 배치 학습 후 손실값은  5.623685280918761
90 배치 학습 후 손실값은  2.4042199502393347
100 배치 학습 후 손실값은  2.986821741753142
100 배치 학습 후 검증 데이터에 대한 정확도:  87.91%
110 배치 학습 후 손실값은  8.836981848855352
120 배치 학습 후 손실값은  3.639570441221318
130 배치 학습 후 손실값은  6.216984495854262
140 배치 학습 후 손실값은  3.7973158385797134
150 배치 학습 후 손실값은  7.859387262749923
160 배치 학습 후 손실값은  3.5849229528887534
170 배치 학습 후 손실값은  4.091073797828593
180 배치 학습 후 손실값은  4.592753150922942
190 배치 학습 후 손실값은  7.3917443078928375
200 배치 학습 후 손실값은  4.835527177705351
200 배치 학습 후 검증 데이터에 대한 정확도:  85.92%
210 배치 학습 후 손실값은  5.003539224241831
220 배치 학습 후 손실값은  5.52621391983965
230 배치 학습 후 손실값은  3.9249624168828134
240 배치 학습 후 손실값은  2.0723265950087377
250 배치 학습 후 손실값은  4.8354287085135

In [15]:
calc_accuracy_model(model, X_test_conv)

검증 데이터에 대한 모델의 정확도 : 
    91.28%
